# Langchain

You will learn about the following things:

* Openai Chatgpt API
* Chatgpt API through Langchain
* Create effective prompt
* Extract infromation from user's query 

### Python packages

You need to install the following Python packages

* langchain
* openai

`pip install langchain openai`

In [2]:
import openai
import os
from dotenv import find_dotenv, load_dotenv

### Set the OpenAI API key

To use the `oepnai` Python package we need to provide OpenAI API key, you can [Get your API key](https://platform.openai.com/account/api-keys).

In [5]:
load_dotenv(find_dotenv())

True

In [6]:
openai.api_key = os.environ['OPENAI_API_KEY']

#### Call Chatgpt API

In [7]:
def chat_completion(prompt, model='gpt-3.5-turbo'):
    messages = [{'role': 'user', 'content': prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message['content']

In [8]:
chat_completion('what is sos?')

'SOS is a Morse code distress signal used internationally to indicate an emergency situation. It consists of three dots, three dashes, and three dots, with no spaces between them. The letters "SOS" do not actually stand for anything, but were chosen because they are easy to transmit and recognize in Morse code.'

#### Prompt engineering

In [41]:
review = '''"The Courage to Be Disliked" by Ichiro Kishimi and Fumitake Koga is a thought-provoking \
and transformative book that challenges conventional wisdom about happiness, interpersonal \
relationships, and personal growth. Drawing on the principles of Alfred Adler, a renowned \
psychologist, this book presents a refreshing and liberating perspective on life.'''

style = '''American English in a calm and respectful tone'''

In [23]:
prompt = f'''Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \

text: ```{review}```'''

print(prompt)

Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone. 
text: ```
"The Courage to Be Disliked" by Ichiro Kishimi and Fumitake Koga is a thought-provoking and transformative book that challenges conventional wisdom about happiness, interpersonal relationships, and personal growth. Drawing on the principles of Alfred Adler, a renowned psychologist, this book presents a refreshing and liberating perspective on life.
```



In [24]:
response = chat_completion(prompt)

In [25]:
response

'"The Courage to Be Disliked" by Ichiro Kishimi and Fumitake Koga is a truly thought-provoking and transformative book that challenges conventional wisdom about happiness, interpersonal relationships, and personal growth. Drawing on the principles of Alfred Adler, a renowned psychologist, this book presents a refreshing and liberating perspective on life.'

### Langchain

In [26]:
from langchain.chat_models import ChatOpenAI

In [27]:
chat = ChatOpenAI(temperature=0.0)

In [28]:
chat.predict(prompt)

'"The Courage to Be Disliked" by Ichiro Kishimi and Fumitake Koga is a book that offers a unique and insightful perspective on happiness, interpersonal relationships, and personal growth. Based on the principles of renowned psychologist Alfred Adler, this thought-provoking and transformative work challenges conventional wisdom and provides a refreshing and liberating approach to life.'

### Prompt engineering

In [45]:
template_string = '''Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```'''

In [46]:
from langchain.prompts import ChatPromptTemplate

In [47]:
prompt_template = ChatPromptTemplate.from_template(
    template=template_string
)

In [48]:
prompt_template

ChatPromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], output_parser=None, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```', template_format='f-string', validate_template=True), additional_kwargs={})])

In [49]:
review = '''"The Courage to Be Disliked" by Ichiro Kishimi and Fumitake Koga is a thought-provoking \
and transformative book that challenges conventional wisdom about happiness, interpersonal \
relationships, and personal growth. Drawing on the principles of Alfred Adler, a renowned \
psychologist, this book presents a refreshing and liberating perspective on life.'''

style = '''American English in a calm and respectful tone'''

In [52]:
review_messages = prompt_template.format_messages(style=style, text=review)

In [53]:
review_messages

[HumanMessage(content='Translate the text that is delimited by triple backticks into a style that is American English in a calm and respectful tone. text: ```"The Courage to Be Disliked" by Ichiro Kishimi and Fumitake Koga is a thought-provoking and transformative book that challenges conventional wisdom about happiness, interpersonal relationships, and personal growth. Drawing on the principles of Alfred Adler, a renowned psychologist, this book presents a refreshing and liberating perspective on life.```', additional_kwargs={}, example=False)]

In [54]:
review_response = chat(review_messages)

In [55]:
review_response

AIMessage(content='"The Courage to Be Disliked" by Ichiro Kishimi and Fumitake Koga is a truly thought-provoking and transformative book that challenges conventional wisdom about happiness, interpersonal relationships, and personal growth. Drawing on the principles of Alfred Adler, a renowned psychologist, this book presents a refreshing and liberating perspective on life that is sure to inspire readers.', additional_kwargs={}, example=False)

### Extract information

In [62]:
user_query = '''I need and iphone 13 max pro 128gb in red color'''

#### First way

In [72]:
first_template = '''For the following text, extract the following information:

color: Was the color of the product. If not found, output -1.

product: Was the product. If this information is not found, output -1.

space: Was the memory space on the product, If this information is not found, output -1.

text: {text}'''

In [73]:
from langchain.prompts import ChatPromptTemplate

In [74]:
prompt_template = ChatPromptTemplate.from_template(first_template)
print(prompt_template)

input_variables=['text'] output_parser=None partial_variables={} messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='For the following text, extract the following information:\n\ncolor: Was the color of the product. If not found, output -1.\n\nproduct: Was the product. If this information is not found, output -1.\n\nspace: Was the memory space on the product, If this information is not found, output -1.\n\ntext: {text}', template_format='f-string', validate_template=True), additional_kwargs={})]


In [75]:
messages = prompt_template.format_messages(text=user_query, format_instructions=format_instructions)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

color: red
product: iPhone 13 Max Pro
space: 128gb


In [76]:
print(type(response.content))

<class 'str'>


#### Second way

In [77]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser

In [79]:
color_schema = ResponseSchema(
    name='color',
    description='Was the color of the product. If not found, output -1.'
)

product_schema = ResponseSchema(
    name='product',
    description='Was the product. If this information is not found, output -1.'
)

space_schema = ResponseSchema(
    name='space',
    description='Was the memory space on the product, If this information is not found, output -1.'
)

response_schemas = [color_schema,
                    product_schema,
                    space_schema]

In [80]:
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

In [81]:
output_parser

StructuredOutputParser(response_schemas=[ResponseSchema(name='color', description='Was the color of the product. If not found, output -1.'), ResponseSchema(name='product', description='Was the product. If this information is not found, output -1.'), ResponseSchema(name='space', description='Was the memory space on the product, If this information is not found, output -1.')])

In [82]:
format_instructions = output_parser.get_format_instructions()

In [83]:
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\\`\\`\\`json" and "\\`\\`\\`":\n\n```json\n{\n\t"color": string  // Was the color of the product. If not found, output -1.\n\t"product": string  // Was the product. If this information is not found, output -1.\n\t"space": string  // Was the memory space on the product, If this information is not found, output -1.\n}\n```'

In [84]:
second_template = '''For the following text, extract the following information:

color: Was the color of the product. If not found, output -1.

product: Was the product. If this information is not found, output -1.

space: Was the memory space on the product, If this information is not found, output -1.

text: {text}
{format_instructions}'''

In [85]:
prompt = ChatPromptTemplate.from_template(template=second_template)

In [90]:
messages = prompt.format_messages(text=user_query, 
                                format_instructions=format_instructions)

In [91]:
messages

[HumanMessage(content='For the following text, extract the following information:\n\ncolor: Was the color of the product. If not found, output -1.\n\nproduct: Was the product. If this information is not found, output -1.\n\nspace: Was the memory space on the product, If this information is not found, output -1.\n\ntext: I need and iphone 13 max pro 128gb in red color\nThe output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\\`\\`\\`json" and "\\`\\`\\`":\n\n```json\n{\n\t"color": string  // Was the color of the product. If not found, output -1.\n\t"product": string  // Was the product. If this information is not found, output -1.\n\t"space": string  // Was the memory space on the product, If this information is not found, output -1.\n}\n```', additional_kwargs={}, example=False)]

In [92]:
response = chat(messages)

In [93]:
print(response.content)

```json
{
	"color": "red",
	"product": "iphone 13 max pro",
	"space": "128gb"
}
```


In [94]:
print(type(response.content))

<class 'str'>


In [95]:
output_dict = output_parser.parse(response.content)

In [96]:
print(output_dict)

{'color': 'red', 'product': 'iphone 13 max pro', 'space': '128gb'}


In [97]:
print(type(output_dict))

<class 'dict'>
